In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score,roc_auc_score,roc_curve,classification_report
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.linear_model import LassoCV, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint as sp_randint
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.metrics import cohen_kappa_score
from sklearn import svm


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df=pd.read_excel("df2.xlsx")
#df=pd.read_excel("C:/Users/priyanka/Desktop/Priya_dse/capstone/df2.xlsx")

df.shape

(118898, 27)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118898 entries, 0 to 118897
Data columns (total 27 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           118898 non-null  object 
 1   is_canceled                     118898 non-null  int64  
 2   lead_time                       118898 non-null  int64  
 3   arrival_date_year               118898 non-null  int64  
 4   arrival_date_month              118898 non-null  int64  
 5   arrival_date_day_of_month       118898 non-null  int64  
 6   stays_in_weekend_nights         118898 non-null  int64  
 7   stays_in_week_nights            118898 non-null  int64  
 8   adults                          118898 non-null  int64  
 9   children                        118898 non-null  int64  
 10  meal                            118898 non-null  object 
 11  country                         118898 non-null  object 
 12  market_segment  

Feature Engineering

In [5]:
df1=df.drop(columns=['stays_in_weekend_nights','stays_in_week_nights'],axis=1)

In [6]:
df2=df1.copy()

In [7]:
df2['room_change'] = np.where(df1['reserved_room_type']==df1['assigned_room_type'],'same_room','room_changed')

In [8]:
df2.drop(columns=['reserved_room_type','assigned_room_type'],axis=1,inplace=True)

In [9]:
df2['total_days'] = np.where(df2['total_days']<8,'Less than 1 Week','More than 1 Week')

In [10]:
df2['adults'] = np.where(df2['adults']==2,'couple','non-couple')

In [11]:
df2['children'] = np.where(df2['children']==0,'no-child','child')

In [12]:
df2['booking_changes'] = np.where(df2['booking_changes']==0,'no_change','change')

In [13]:
df2['total_of_special_requests'] = np.where(df2['total_of_special_requests']==0,'Yes','No')

In [14]:
df2.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,adults,children,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_days,room_change
0,Resort Hotel,0,342,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,0,0,0,change,No Deposit,0,Transient,0.0,0,Yes,Less than 1 Week,same_room
1,Resort Hotel,0,737,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,0,0,0,change,No Deposit,0,Transient,0.0,0,Yes,Less than 1 Week,same_room
2,Resort Hotel,0,7,2015,7,1,non-couple,no-child,BB,GBR,Direct,Direct,0,0,0,no_change,No Deposit,0,Transient,75.0,0,Yes,Less than 1 Week,room_changed
3,Resort Hotel,0,13,2015,7,1,non-couple,no-child,BB,GBR,Corporate,Corporate,0,0,0,no_change,No Deposit,0,Transient,75.0,0,Yes,Less than 1 Week,same_room
4,Resort Hotel,0,14,2015,7,1,couple,no-child,BB,GBR,Online TA,TA/TO,0,0,0,no_change,No Deposit,0,Transient,98.0,0,No,Less than 1 Week,same_room


In [15]:
df2['check_in']=df2.previous_bookings_not_canceled

In [16]:
df2.drop('previous_bookings_not_canceled',axis=1,inplace=True)

In [17]:
df2['total_bookings']=df2['previous_cancellations']+df2['check_in']

In [18]:
df2['cancellation_percent']=((df2.previous_cancellations)/(df2.total_bookings))*100

In [19]:
#there will be nan values because, there are new customers who would have made  booking,
#hence the previous cancellation will be 0 for them and cha

df2['cancellation_percent'].isnull().sum()

109684

In [20]:
df2.cancellation_percent.fillna(0,inplace=True)

In [21]:
df2.isnull().sum()

hotel                          0
is_canceled                    0
lead_time                      0
arrival_date_year              0
arrival_date_month             0
arrival_date_day_of_month      0
adults                         0
children                       0
meal                           0
country                        0
market_segment                 0
distribution_channel           0
is_repeated_guest              0
previous_cancellations         0
booking_changes                0
deposit_type                   0
days_in_waiting_list           0
customer_type                  0
adr                            0
required_car_parking_spaces    0
total_of_special_requests      0
total_days                     0
room_change                    0
check_in                       0
total_bookings                 0
cancellation_percent           0
dtype: int64

In [22]:
df2['required_car_parking_spaces'] = np.where(df2['required_car_parking_spaces']==0,'No','Yes')

In [23]:
df2['is_repeated_guest'] = np.where(df2['is_repeated_guest']==0,'No','Yes')

In [24]:
df2.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,adults,children,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_days,room_change,check_in,total_bookings,cancellation_percent
0,Resort Hotel,0,342,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,No,0,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0,0.0
1,Resort Hotel,0,737,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,No,0,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0,0.0
2,Resort Hotel,0,7,2015,7,1,non-couple,no-child,BB,GBR,Direct,Direct,No,0,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,room_changed,0,0,0.0
3,Resort Hotel,0,13,2015,7,1,non-couple,no-child,BB,GBR,Corporate,Corporate,No,0,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,same_room,0,0,0.0
4,Resort Hotel,0,14,2015,7,1,couple,no-child,BB,GBR,Online TA,TA/TO,No,0,no_change,No Deposit,0,Transient,98.0,No,No,Less than 1 Week,same_room,0,0,0.0


In [25]:
pip install country_converter

     |████████████████████████████████| 51kB 3.3MB/s 
  Created wheel for country-converter: filename=country_converter-0.6.7-cp36-none-any.whl size=50216 sha256=ce60d66269107e82edf12604b7b2a6f24bf491a3caff51685d7421648edc198c
  Stored in directory: /root/.cache/pip/wheels/f9/4a/e7/c8ef69407e41021c414dd56e50bdac424efd1bca3a4346eeb1
Successfully built country-converter


In [26]:
import country_converter as coco
cc = coco.CountryConverter()
continents = []
for index, row in df2.iterrows():
    continent = cc.convert([row.country], to='continent')
    continents.append(continent)
cont_df = pd.DataFrame(continents, columns=['continent'])
df2 = pd.concat([df2, cont_df], 1)


In [27]:
df2.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,adults,children,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_days,room_change,check_in,total_bookings,cancellation_percent,continent
0,Resort Hotel,0,342,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,No,0,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0,0.0,Europe
1,Resort Hotel,0,737,2015,7,1,couple,no-child,BB,PRT,Direct,Direct,No,0,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0,0.0,Europe
2,Resort Hotel,0,7,2015,7,1,non-couple,no-child,BB,GBR,Direct,Direct,No,0,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,room_changed,0,0,0.0,Europe
3,Resort Hotel,0,13,2015,7,1,non-couple,no-child,BB,GBR,Corporate,Corporate,No,0,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,same_room,0,0,0.0,Europe
4,Resort Hotel,0,14,2015,7,1,couple,no-child,BB,GBR,Online TA,TA/TO,No,0,no_change,No Deposit,0,Transient,98.0,No,No,Less than 1 Week,same_room,0,0,0.0,Europe


In [28]:
df2.drop(columns=['country','previous_cancellations','check_in'],axis=1,inplace=True)

In [29]:
df2.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,adults,children,meal,market_segment,distribution_channel,is_repeated_guest,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,total_days,room_change,total_bookings,cancellation_percent,continent
0,Resort Hotel,0,342,2015,7,1,couple,no-child,BB,Direct,Direct,No,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0.0,Europe
1,Resort Hotel,0,737,2015,7,1,couple,no-child,BB,Direct,Direct,No,change,No Deposit,0,Transient,0.0,No,Yes,Less than 1 Week,same_room,0,0.0,Europe
2,Resort Hotel,0,7,2015,7,1,non-couple,no-child,BB,Direct,Direct,No,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,room_changed,0,0.0,Europe
3,Resort Hotel,0,13,2015,7,1,non-couple,no-child,BB,Corporate,Corporate,No,no_change,No Deposit,0,Transient,75.0,No,Yes,Less than 1 Week,same_room,0,0.0,Europe
4,Resort Hotel,0,14,2015,7,1,couple,no-child,BB,Online TA,TA/TO,No,no_change,No Deposit,0,Transient,98.0,No,No,Less than 1 Week,same_room,0,0.0,Europe


Statistical Test:

In [30]:
# Stat test for categorical columns

cat = df2.select_dtypes(include='object')
alpha = 0.05
for i in cat.columns:
    table = pd.crosstab(cat[i],df2['is_canceled'])
    stat,p,dof,expected = st.chi2_contingency(table)
    prob = 0.95
    critical = st.chi2.ppf(prob,dof)
    print(i,": ")
    print("stat:",stat)
    print("critical:",critical)
    print("dof:",dof)
    print("pvalue:",p)
    if abs(stat)>=critical:
        print('it is dependent reject H0')
        print()
    else :
        print('it is independent fail to reject H0')
        print()

hotel : 
stat: 2133.2083285446906
critical: 3.841458820694124
dof: 1
pvalue: 0.0
it is dependent reject H0

adults : 
stat: 757.5331998155203
critical: 3.841458820694124
dof: 1
pvalue: 9.234715823979752e-167
it is dependent reject H0

children : 
stat: 1.8941522353498146
critical: 3.841458820694124
dof: 1
pvalue: 0.16873433313656133
it is independent fail to reject H0

meal : 
stat: 304.1985042024184
critical: 9.487729036781154
dof: 4
pvalue: 1.3461925413085518e-64
it is dependent reject H0

market_segment : 
stat: 8392.685917219074
critical: 12.591587243743977
dof: 6
pvalue: 0.0
it is dependent reject H0

distribution_channel : 
stat: 3650.453219706919
critical: 9.487729036781154
dof: 4
pvalue: 0.0
it is dependent reject H0

is_repeated_guest : 
stat: 861.663820464543
critical: 3.841458820694124
dof: 1
pvalue: 2.1176165683492342e-189
it is dependent reject H0

booking_changes : 
stat: 4171.172986893723
critical: 3.841458820694124
dof: 1
pvalue: 0.0
it is dependent reject H0

deposit_t

In [31]:
# Stat test for numerical columns

num = df2.select_dtypes(include="number")
alpha = 0.05
for i in num.columns:
    t_stat = st.ttest_ind(num[i],df2['is_canceled'])
    dof=(len(num[i])-1)+(len(df2['is_canceled'])-1)
    t_crit = st.t.isf(alpha,dof)
    p_value = st.t.sf(t_stat,dof)
    print(i,": ")
    print("t_stat",t_stat)
    print("dof",dof)
    print("t_crit",t_crit)
    print("p_value",p_value)
    if (t_stat[0])>=t_crit:
        print('it is dependent reject H0')
        print()
    else :
        print('it is independent fail to reject H0')
        print() 
                          

is_canceled : 
t_stat Ttest_indResult(statistic=0.0, pvalue=1.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.5        0.15865576]
it is independent fail to reject H0

lead_time : 
t_stat Ttest_indResult(statistic=335.25460306713217, pvalue=0.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.  0.5]
it is dependent reject H0

arrival_date_year : 
t_stat Ttest_indResult(statistic=811331.940902273, pvalue=0.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.  0.5]
it is dependent reject H0

arrival_date_month : 
t_stat Ttest_indResult(statistic=682.1573045490128, pvalue=0.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.  0.5]
it is dependent reject H0

arrival_date_day_of_month : 
t_stat Ttest_indResult(statistic=605.0242611092099, pvalue=0.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.  0.5]
it is dependent reject H0

days_in_waiting_list : 
t_stat Ttest_indResult(statistic=38.30755038655065, pvalue=0.0)
dof 237794
t_crit 1.6448600349143314
p_value [0.  0.5]
it is dependent r

creating dummies 

In [32]:
df3=df2.copy()
df3.shape

(118898, 24)

In [33]:
for i in df3.select_dtypes(include='object').columns:
    df3 = pd.get_dummies(df3,drop_first = True)

In [34]:
df3.shape

(118898, 43)

In [35]:
df3.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,days_in_waiting_list,adr,total_bookings,cancellation_percent,hotel_Resort Hotel,adults_non-couple,children_no-child,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,is_repeated_guest_Yes,booking_changes_no_change,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,required_car_parking_spaces_Yes,total_of_special_requests_Yes,total_days_More than 1 Week,room_change_same_room,continent_America,continent_Antarctica,continent_Asia,continent_Europe,continent_Oceania,continent_not found
0,0,342,2015,7,1,0,0.0,0,0.0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
1,0,737,2015,7,1,0,0.0,0,0.0,1,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
2,0,7,2015,7,1,0,75.0,0,0.0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0
3,0,13,2015,7,1,0,75.0,0,0.0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0
4,0,14,2015,7,1,0,98.0,0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0


Model Building

In [36]:
x=df3.drop('is_canceled',axis=1)
y=df3.is_canceled
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=1)

In [37]:
def model(algo,xtrain,ytrain,xtest,ytest):
    algo.fit(xtrain,ytrain)
    
    ytrain_prob=algo.predict_proba(xtrain)[:,1]
    ytrain_pred=algo.predict(xtrain)

    ytest_prob=algo.predict_proba(xtest)[:,1]
    ytest_pred=algo.predict(xtest)

    print("accuracy for train :",accuracy_score(ytrain,ytrain_pred))
    print("accuracy for test :",accuracy_score(ytest,ytest_pred))
    print()

    print("confusion matrix for train :")
    print(confusion_matrix(ytrain,ytrain_pred))
    print()
    print("confusion matrix for test :")
    print(confusion_matrix(ytest,ytest_pred))
    print()

    print("AUC for train : ",roc_auc_score(ytrain,ytrain_prob))
    print("AUC for test : ",roc_auc_score(ytest,ytest_prob))

    print()
    print("classification report :")
    print(classification_report(ytest,ytest_pred))

    print()
    cohen_score = cohen_kappa_score(ytest, ytest_pred)
    print("Cohen Kappa Score:",cohen_score)
    

In [ ]:
#Logistic :

model(LogisticRegression(fit_intercept=True,solver='liblinear'),xtrain,ytrain,xtest,ytest)

accuracy for train : 0.8126111404815687
accuracy for test : 0.807793664143538

confusion matrix for train :
[[47895  4382]
 [11214 19737]]

confusion matrix for test :
[[20515  1953]
 [ 4903  8299]]

AUC for train :  0.8717341167593406
AUC for test :  0.8661885150897638

classification report :
              precision    recall  f1-score   support

           0       0.81      0.91      0.86     22468
           1       0.81      0.63      0.71     13202

    accuracy                           0.81     35670
   macro avg       0.81      0.77      0.78     35670
weighted avg       0.81      0.81      0.80     35670


Cohen Kappa Score: 0.5678582120067592


In [38]:
# Decision Tree:

model(DecisionTreeClassifier(),xtrain,ytrain,xtest,ytest)

accuracy for train : 0.9916374297111549
accuracy for test : 0.8243622091393328

confusion matrix for train :
[[52089   188]
 [  508 30443]]

confusion matrix for test :
[[19243  3225]
 [ 3040 10162]]

AUC for train :  0.9998128023855832
AUC for test :  0.8165136633448511

classification report :
              precision    recall  f1-score   support

           0       0.86      0.86      0.86     22468
           1       0.76      0.77      0.76     13202

    accuracy                           0.82     35670
   macro avg       0.81      0.81      0.81     35670
weighted avg       0.82      0.82      0.82     35670


Cohen Kappa Score: 0.6243901780155772


In [39]:
## Hyper parameter tuning for Decesion Tree 

dtc1=DecisionTreeClassifier()
max_depth=[2,50]
min_samples_split=[2,200]
min_sample_leaf=[1,150]
criteria=['gini','entropy']

param={'max_depth':max_depth,
        'min_samples_split': min_samples_split,
        'min_samples_leaf' : min_sample_leaf,
        'criterion' : criteria}

gsearch=GridSearchCV(dtc1,param_grid=param,scoring='roc_auc',cv=2)#, n_jobs=-1)
gsearch.fit(x,y)

GridSearchCV(cv=2, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [2, 50

In [40]:
print(gsearch.best_params_)
print()

model(DecisionTreeClassifier(**gsearch.best_params_),xtrain,ytrain,xtest,ytest)

{'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 150, 'min_samples_split': 2}

accuracy for train : 0.7680588263565147
accuracy for test : 0.7666105971404542

confusion matrix for train :
[[52194    83]
 [19221 11730]]

confusion matrix for test :
[[22434    34]
 [ 8291  4911]]

AUC for train :  0.6889686984504972
AUC for test :  0.6854923828174675

classification report :
              precision    recall  f1-score   support

           0       0.73      1.00      0.84     22468
           1       0.99      0.37      0.54     13202

    accuracy                           0.77     35670
   macro avg       0.86      0.69      0.69     35670
weighted avg       0.83      0.77      0.73     35670


Cohen Kappa Score: 0.4253295364471714


In [41]:
## Random forest :

model(RandomForestClassifier(n_estimators=10),xtrain,ytrain,xtest,ytest)

accuracy for train : 0.9802830778103523
accuracy for test : 0.8574151948416036

confusion matrix for train :
[[51903   374]
 [ 1267 29684]]

confusion matrix for test :
[[20769  1699]
 [ 3387  9815]]

AUC for train :  0.9982513819296117
AUC for test :  0.9154708915980679

classification report :
              precision    recall  f1-score   support

           0       0.86      0.92      0.89     22468
           1       0.85      0.74      0.79     13202

    accuracy                           0.86     35670
   macro avg       0.86      0.83      0.84     35670
weighted avg       0.86      0.86      0.86     35670


Cohen Kappa Score: 0.6859135326305461


In [43]:
## Hype parameter tuning for random forest

rfc=RandomForestClassifier(n_estimators=10)
params={
    'n_estimators':sp_randint(5,150),
    'max_features':sp_randint(1,20),
    'max_depth': sp_randint(2,10),
    'min_samples_leaf':sp_randint(1,50),
    'min_samples_split':sp_randint(2,50),
    'criterion': ['gini','entropy'] }

rsearch_rf=RandomizedSearchCV(rfc,param_distributions=params,n_iter=100,cv=3,n_jobs=-1,scoring='roc_auc')
rsearch_rf.fit(x,y)

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [44]:
print(rsearch_rf.best_params_)
print()

model(RandomForestClassifier(**rsearch_rf.best_params_),xtrain,ytrain,xtest,ytest)


{'criterion': 'entropy', 'max_depth': 6, 'max_features': 8, 'min_samples_leaf': 9, 'min_samples_split': 38, 'n_estimators': 27}

accuracy for train : 0.8164319700100927
accuracy for test : 0.8128118867395571

confusion matrix for train :
[[49727  2550]
 [12728 18223]]

confusion matrix for test :
[[21324  1144]
 [ 5533  7669]]

AUC for train :  0.8753941062138821
AUC for test :  0.8692412989820841

classification report :
              precision    recall  f1-score   support

           0       0.79      0.95      0.86     22468
           1       0.87      0.58      0.70     13202

    accuracy                           0.81     35670
   macro avg       0.83      0.76      0.78     35670
weighted avg       0.82      0.81      0.80     35670


Cohen Kappa Score: 0.5689853843575186


In [45]:
## ADA BOOST :

model(AdaBoostClassifier(random_state=1),xtrain,ytrain,xtest,ytest)


accuracy for train : 0.8188350074494161
accuracy for test : 0.8147182506307822

confusion matrix for train :
[[48761  3516]
 [11562 19389]]

confusion matrix for test :
[[20902  1566]
 [ 5043  8159]]

AUC for train :  0.877875762517297
AUC for test :  0.8724884696555895

classification report :
              precision    recall  f1-score   support

           0       0.81      0.93      0.86     22468
           1       0.84      0.62      0.71     13202

    accuracy                           0.81     35670
   macro avg       0.82      0.77      0.79     35670
weighted avg       0.82      0.81      0.81     35670


Cohen Kappa Score: 0.5798010609788806


In [46]:
## Light GBM :

model(lgb.LGBMClassifier(),xtrain,ytrain,xtest,ytest)


accuracy for train : 0.8484764742634691
accuracy for test : 0.8416316232127838

confusion matrix for train :
[[48755  3522]
 [ 9089 21862]]

confusion matrix for test :
[[20867  1601]
 [ 4048  9154]]

AUC for train :  0.9221652061837469
AUC for test :  0.9113714812282503

classification report :
              precision    recall  f1-score   support

           0       0.84      0.93      0.88     22468
           1       0.85      0.69      0.76     13202

    accuracy                           0.84     35670
   macro avg       0.84      0.81      0.82     35670
weighted avg       0.84      0.84      0.84     35670


Cohen Kappa Score: 0.6468454058062555


In [47]:
#SVM Model

model(svm.SVC(probability=True),xtrain,ytrain,xtest,ytest)



accuracy for train : 0.6830033161916663
accuracy for test : 0.6851135407905803

confusion matrix for train :
[[51917   360]
 [26023  4928]]

confusion matrix for test :
[[22337   131]
 [11101  2101]]

AUC for train :  0.6886482059592504
AUC for test :  0.6926824113593312

classification report :
              precision    recall  f1-score   support

           0       0.67      0.99      0.80     22468
           1       0.94      0.16      0.27     13202

    accuracy                           0.69     35670
   macro avg       0.80      0.58      0.54     35670
weighted avg       0.77      0.69      0.60     35670


Cohen Kappa Score: 0.18501252525818634


In [51]:
#CAT BOOST
pip install catboost

     |████████████████████████████████| 64.8MB 65kB/s 


In [52]:
from catboost import Pool, CatBoostClassifier

In [53]:
model(CatBoostClassifier(random_state=1),xtrain,ytrain,xtest,ytest)

Learning rate set to 0.06806
0:	learn: 0.6403244	total: 89.6ms	remaining: 1m 29s
1:	learn: 0.5992636	total: 126ms	remaining: 1m 2s
2:	learn: 0.5614898	total: 162ms	remaining: 54s
3:	learn: 0.5314057	total: 203ms	remaining: 50.5s
4:	learn: 0.5089793	total: 240ms	remaining: 47.7s
5:	learn: 0.4915130	total: 277ms	remaining: 45.8s
6:	learn: 0.4752480	total: 316ms	remaining: 44.8s
7:	learn: 0.4645901	total: 353ms	remaining: 43.7s
8:	learn: 0.4563533	total: 391ms	remaining: 43s
9:	learn: 0.4494172	total: 431ms	remaining: 42.7s
10:	learn: 0.4400248	total: 469ms	remaining: 42.1s
11:	learn: 0.4349267	total: 507ms	remaining: 41.7s
12:	learn: 0.4308166	total: 543ms	remaining: 41.2s
13:	learn: 0.4266708	total: 581ms	remaining: 40.9s
14:	learn: 0.4222756	total: 618ms	remaining: 40.6s
15:	learn: 0.4188330	total: 658ms	remaining: 40.5s
16:	learn: 0.4162066	total: 697ms	remaining: 40.3s
17:	learn: 0.4131081	total: 735ms	remaining: 40.1s
18:	learn: 0.4093877	total: 774ms	remaining: 40s
19:	learn: 0.407

In [54]:
## XG Boost
pip install xgboost

SyntaxError: ignored

In [55]:
pip install xgboost

In [56]:
import xgboost as xgb

model(xgb.XGBClassifier(random_state=1),xtrain,ytrain,xtest,ytest)

accuracy for train : 0.8197721920507521
accuracy for test : 0.8156433978132884

confusion matrix for train :
[[49065  3212]
 [11788 19163]]

confusion matrix for test :
[[21034  1434]
 [ 5142  8060]]

AUC for train :  0.8908243538375494
AUC for test :  0.8861929627626136

classification report :
              precision    recall  f1-score   support

           0       0.80      0.94      0.86     22468
           1       0.85      0.61      0.71     13202

    accuracy                           0.82     35670
   macro avg       0.83      0.77      0.79     35670
weighted avg       0.82      0.82      0.81     35670


Cohen Kappa Score: 0.5802979359982816


In [57]:
## KNN

from sklearn.neighbors import KNeighborsClassifier
# knc = KNeighborsClassifier()
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [58]:
xtrain1,xtest1,ytrain1,ytest1=train_test_split(x,y,test_size=0.3,random_state=1)

In [59]:
xtrain_ss=ss.fit_transform(xtrain1)
xtest_ss=ss.transform(xtest1)

In [60]:
model(KNeighborsClassifier(),xtrain_ss,ytrain1,xtest_ss,ytest1)

# knc.fit(xtrain_ss,ytrain1)

# ytrain_prob=knc.predict_proba(xtrain_ss)[:,1]
# ytrain_pred=knc.predict(xtrain_ss)

# ytest_prob=knc.predict_proba(xtest_ss)[:,1]
# ytest_pred=knc.predict(xtest_ss)

# print("accuracy for train :",accuracy_score(ytrain1,ytrain_pred))
# print("accuracy for test :",accuracy_score(ytest1,ytest_pred))
# print()

# print("confusion matrix for train :")
# print(confusion_matrix(ytrain1,ytrain_pred))
# print()
# print("confusion matrix for test :")
# print(confusion_matrix(ytest1,ytest_pred))
# print()

# print("AUC for train : ",roc_auc_score(ytrain1,ytrain_prob))
# print("AUC for test : ",roc_auc_score(ytest1,ytest_prob))


# print()
# print("classification report :")
# print(classification_report(ytest1,ytest_pred))


accuracy for train : 0.881061661940693
accuracy for test : 0.8265769554247266

confusion matrix for train :
[[48303  3974]
 [ 5925 25026]]

confusion matrix for test :
[[19790  2678]
 [ 3508  9694]]

AUC for train :  0.9550058189536721
AUC for test :  0.8862736579124926

classification report :
              precision    recall  f1-score   support

           0       0.85      0.88      0.86     22468
           1       0.78      0.73      0.76     13202

    accuracy                           0.83     35670
   macro avg       0.82      0.81      0.81     35670
weighted avg       0.83      0.83      0.83     35670


Cohen Kappa Score: 0.6231696650256984
